# Backtracking, solution and maze generation

# Context
## The HW adopts the following plan:
1. Solution of labyrinths by backtracking
2. Generation of labyrinths by backtracking in recursive version
3. Generation of labyrinths by backtracking in iterative version
4. Generation of uniformly distributed mazes with Wilson’s algorithm

In [1]:
import java.util.ArrayList;
import java.util.List;
import java.util.stream.Collectors;
import java.util.LinkedList;
import java.util.Random;

import java.awt.BasicStroke;
import java.awt.BorderLayout;
import java.awt.Color;
import java.awt.Dimension;
import java.awt.Graphics;
import java.awt.Graphics2D;
import java.awt.event.KeyEvent;
import java.awt.event.KeyListener;

import java.io.IOException;

import java.nio.charset.StandardCharsets;
import java.nio.file.Files;
import java.nio.file.Paths;

import javax.swing.JFrame;
import javax.swing.JPanel;

In [2]:
// Cell 1: Forward Declaration of Maze

// Placeholder Maze class to resolve dependencies
class Maze {
    // Empty placeholder; full definition in next cell
}

In [3]:
// Cell 2: Core Class Definitions with GUI and File Support
// Cell Abstract Class
public abstract class Cell {
    private boolean isMarked;
    private boolean isExit;
    private ArrayList<Cell> neighbors;
    private ArrayList<Boolean> walls;
    protected Maze maze;

    Cell() {}
    
    Cell(Maze maze) {
        isMarked = false;
        isExit = false;
        neighbors = new ArrayList<>();
        walls = new ArrayList<>();
        this.maze = maze;
    }

    public String toString() {
        String s = "Case ";
        if (!isMarked) s += "no ";
        s += "marked ";
        int n = getNeighbors(true).size();
        s += " with " + n + " neighbors whose ";
        n = getNeighbors(false).size();
        s += n + " accessible.";
        return s;
    }

    java.util.List<Cell> getNeighbors(boolean ignoreWalls) {
        if (ignoreWalls) return new ArrayList<>(neighbors);
        else return neighbors.stream().filter(cell -> this.hasPassageTo(cell)).collect(Collectors.toList());
    }

    boolean hasPassageTo(Cell c) {
        if (c == null || neighbors.indexOf(c) == -1 || c.neighbors.indexOf(this) == -1) return false;
        return !walls.get(neighbors.indexOf(c));
    }

    void breakWall(Cell c) {
        if (c == null || neighbors.indexOf(c) == -1 || c.neighbors.indexOf(this) == -1)
            throw new IllegalArgumentException("cells are not neighbors");
        walls.set(neighbors.indexOf(c), false);
        c.walls.set(c.neighbors.indexOf(this), false);
    }

    boolean isMarked() { return isMarked; }
    void setMarked(boolean b) { isMarked = b; }
    boolean isExit() { return isExit; }
    void setExit(boolean b) { isExit = b; }
    boolean isIsolated() { return getNeighbors(false).isEmpty(); }
    void addNeighbor(Cell n) { neighbors.add(n); walls.add(true); }

    abstract boolean searchPath();
    abstract void generateRec();
}

// ExtendedCell Class with Placeholders
class ExtendedCell extends Cell {
    public ExtendedCell(Maze maze) {
        super(maze);
    }

    boolean searchPath() {
        maze.slow();
        System.out.println("searchPath() not implemented yet.");
        return false;
    }

    void generateRec() {
        maze.slow();
        System.out.println("generateRec() not implemented yet.");
    }
}

// Bag Class
class Bag {
    private LinkedList<Cell> list;
    private int method;
    private Random rnd;
    private boolean dirty;
    private int currentIndex;

    static final int NEWEST = 0;
    static final int OLDEST = 1;
    static final int MIDDLE = 2;
    static final int RANDOM = 3;

    Bag(int method) {
        this.list = new LinkedList<>();
        this.method = method;
        if (method == RANDOM) this.rnd = new Random();
        this.dirty = true;
        this.currentIndex = -1;
    }

    boolean isEmpty() { return list.isEmpty(); }
    void add(Cell c) { list.addLast(c); dirty = true; }
    Cell peek() { return list.get(nextIndex()); }
    Cell pop() {
        Cell ret = list.get(nextIndex());
        list.remove(nextIndex());
        dirty = true;
        return ret;
    }

    private int nextIndex() {
        if (!dirty) return currentIndex;
        int bound = list.size();
        switch (method) {
            case NEWEST: currentIndex = bound - 1; break;
            case OLDEST: currentIndex = 0; break;
            case MIDDLE: currentIndex = bound / 2; break;
            case RANDOM: currentIndex = rnd.nextInt(bound); break;
            default: throw new IllegalStateException("invalid selection method");
        }
        dirty = false;
        return currentIndex;
    }
}

// UnionFind Class
class UnionFind {
    private int[] link;
    private int[] rank;
    private int numClasses;
    private int[] size;

    UnionFind(int n) {
        if (n < 0) throw new IllegalArgumentException();
        this.link = new int[n];
        for (int i = 0; i < n; i++) this.link[i] = i;
        this.rank = new int[n];
        this.numClasses = n;
        this.size = new int[n];
        for (int i = 0; i < n; i++) this.size[i] = 1;
    }

    int numClasses() { return this.numClasses; }
    int find(int i) {
        if (i < 0 || i >= link.length) throw new ArrayIndexOutOfBoundsException(i);
        int p = link[i];
        if (p == i) return i;
        int r = find(p);
        link[i] = r;
        return r;
    }
    void union(int i, int j) {
        int ri = find(i);
        int rj = find(j);
        if (ri == rj) return;
        numClasses--;
        if (rank[ri] < rank[rj]) {
            link[ri] = rj;
            size[rj] += size[ri];
        } else {
            link[rj] = ri;
            if (rank[ri] == rank[rj]) rank[ri]++;
            size[ri] += size[rj];
        }
    }
    int getSize(int i) { return size[find(i)]; }
    boolean sameClass(int i, int j) { return find(i) == find(j); }
}

// Coordinate Class
class Coordinate {
    final int i, j;

    Coordinate(int i, int j) {
        this.i = i;
        this.j = j;
    }

    @Override
    public String toString() { return "(" + i + "," + j + ")"; }

    @Override
    public boolean equals(Object o) {
        if (o == null) return false;
        if (!(o instanceof Coordinate)) return false;
        Coordinate that = (Coordinate)o;
        return this.i == that.i && this.j == that.j;
    }

    @Override
    public int hashCode() { return 31 * i + j; }
}

// MazeFrame Class (GUI)
class MazeFrame extends JFrame {
    MazeFrame(Cell[][] grid, int height, int width, int step) {
        MazeWindow window = new MazeWindow(grid, height, width, step);
        this.setTitle("Labyrinthe");
        window.setPreferredSize(new Dimension(width * step + 1, height * step + 1));
        this.add(window, BorderLayout.CENTER);
        this.pack();
        this.add(window);
        this.addKeyListener(window);
        //this.setDefaultCloseOperation(JFrame.EXIT_ON_CLOSE);
        this.setDefaultCloseOperation(JFrame.DISPOSE_ON_CLOSE);
        this.setResizable(false);
        this.setLocationRelativeTo(null);
        this.setVisible(true);
    }
}

// MazeWindow Class (GUI)
class MazeWindow extends JPanel implements KeyListener {
    private final int height, width, step;
    private Cell[][] grid;

    MazeWindow(Cell[][] grid, int height, int width, int step) {
        this.grid = grid;
        this.height = height;
        this.width = width;
        this.step = step;
    }

    @Override
    public void paintComponent(Graphics g) {
        super.paintComponent(g);
        Graphics2D g2d = (Graphics2D) g;
        g2d.setStroke(new BasicStroke(3));
        int w = this.width * this.step;
        int h = this.height * this.step;
        g.drawLine(0, 0, w, 0);
        g.drawLine(0, 0, 0, h);
        g.drawLine(w, 0, w, h);
        g.drawLine(0, h, w, h);

        for (int i = 0; i < height; ++i) {
            for (int j = 0; j < width; ++j) {
                g2d.setColor(Color.BLACK);
                if (i > 0 && !grid[i][j].hasPassageTo(grid[i-1][j]))
                    g2d.drawLine(j*step, i*step, (j+1)*step, i*step);
                if (j < width-1 && !grid[i][j].hasPassageTo(grid[i][j+1]))
                    g2d.drawLine((j+1)*step, i*step, (j+1)*step, (i+1)*step);
                if (i < height-1 && !grid[i][j].hasPassageTo(grid[i+1][j]))
                    g2d.drawLine(j*step, (i+1)*step, (j+1)*step, (i+1)*step);
                if (j > 0 && !grid[i][j].hasPassageTo(grid[i][j-1]))
                    g2d.drawLine(j*step, i*step, j*step, (i+1)*step);

                g2d.setColor(Color.RED);
                if (grid[i][j].isMarked()) {
                    int midx = j*step + step/2;
                    int midy = i*step + step/2;
                    g2d.fillOval(midx - step/4, midy - step/4, step/2, step/2);
                }
            }
        }
    }

    @Override
    public void keyTyped(KeyEvent ev) {    
        if (ev.getKeyChar() == 'q') System.exit(0);
    }

    @Override
    public void keyPressed(KeyEvent ev) {}
    @Override
    public void keyReleased(KeyEvent ev) {}
}

// Maze Class with File Support
public abstract class Maze {
    protected  int height, width;
    protected  Cell[][] grid;
    private MazeFrame frame;
    private static final int step = 20;

    abstract void generateIter(int selectionMethod);

    abstract void generateWilson();

    Cell getCell(int i, int j) {
        if (i < 0 || i >= height || j < 0 || j >= width)
            throw new IllegalArgumentException("invalid indices");
        return grid[i][j];
    }

    Cell getFirstCell() { return getCell(0, 0); }

    int coordToInt(int i, int j) {
        if (i < 0 || i >= height || j < 0 || j >= width)
            throw new IndexOutOfBoundsException();
        return i * width + j;
    }

    Coordinate intToCoord(int x) {
        if (x < 0 || x >= height * width)
            throw new IndexOutOfBoundsException();
        return new Coordinate(x / width, x % width);
    }

    void slow() {
        if (frame == null) return;
        try {
            Thread.sleep(10);
            frame.repaint();
        } catch (InterruptedException e) {}
    }

    Maze(int height, int width) {
        this(height, width, true);
    }

    Maze(int height, int width, boolean window) {
        if (height <= 0 || width <= 0)
            throw new IllegalArgumentException("height and width must be positive");
        this.height = height;
        this.width = width;
        grid = new Cell[height][width];
        for (int i = 0; i < height; ++i)
            for (int j = 0; j < width; ++j)
                grid[i][j] = new ExtendedCell(this);
        for (int i = 0; i < height; ++i) {
            for (int j = 0; j < width; ++j) {
                if (i < height - 1) {
                    grid[i][j].addNeighbor(grid[i+1][j]);
                    grid[i+1][j].addNeighbor(grid[i][j]);
                }
                if (j < width - 1) {
                    grid[i][j].addNeighbor(grid[i][j+1]);
                    grid[i][j+1].addNeighbor(grid[i][j]);
                }
            }
        }
        grid[height-1][width-1].setExit(true);
        if (window) frame = new MazeFrame(grid, height, width, step);
    }

    Maze(String path) throws IOException {
        this(Files.readAllLines(Paths.get(path), StandardCharsets.UTF_8), true); // Default GUI on
    }

    Maze(String path, boolean window) throws IOException {
        this(Files.readAllLines(Paths.get(path), StandardCharsets.UTF_8), window);
    }

    Maze(java.util.List<String> lines) {
        this(lines, true);
    }

    Maze(java.util.List<String> lines, boolean window) {
        if (lines.size() < 2)
            throw new IllegalArgumentException("too few lines");
        this.height = Integer.parseInt(lines.get(0));
        this.width = Integer.parseInt(lines.get(1));
        this.grid = new Cell[height][width];
        for (int i = 0; i < height; ++i)
            for (int j = 0; j < width; ++j)
                grid[i][j] = new ExtendedCell(this);
        for (int i = 0; i < height; ++i) {
            for (int j = 0; j < width; ++j) {
                if (i < height - 1) {
                    grid[i][j].addNeighbor(grid[i+1][j]);
                    grid[i+1][j].addNeighbor(grid[i][j]);
                }
                if (j < width - 1) {
                    grid[i][j].addNeighbor(grid[i][j+1]);
                    grid[i][j+1].addNeighbor(grid[i][j]);
                }
            }
        }
        grid[height-1][width-1].setExit(true);
        int i = 0, j = 0;
        for (String line : lines.subList(2, lines.size())) {
            for (int k = 0; k < line.length(); ++k) {
                switch (line.charAt(k)) {
                    case 'N': grid[i][j].breakWall(grid[i-1][j]); break;
                    case 'E': grid[i][j].breakWall(grid[i][j+1]); break;
                    case 'S': grid[i][j].breakWall(grid[i+1][j]); break;
                    case 'W': grid[i][j].breakWall(grid[i][j-1]); break;
                    case '*': grid[i][j].setMarked(true); break;
                    default: throw new IllegalArgumentException("illegal character");
                }
            }
            ++j;
            if (j >= width) { j = 0; ++i; }
        }
        if (window) frame = new MazeFrame(grid, height, width, step);
    }

    @Override
    public String toString() {
        StringBuilder sb = new StringBuilder();
        sb.append(height).append('\n').append(width).append('\n');
        for (int i = 0; i < height; ++i) {
            for (int j = 0; j < width; ++j) {
                if (i > 0 && grid[i][j].hasPassageTo(grid[i-1][j])) sb.append('N');
                if (j < width-1 && grid[i][j].hasPassageTo(grid[i][j+1])) sb.append('E');
                if (i < height-1 && grid[i][j].hasPassageTo(grid[i+1][j])) sb.append('S');
                if (j > 0 && grid[i][j].hasPassageTo(grid[i][j-1])) sb.append('W');
                if (grid[i][j].isMarked()) sb.append('*');
                sb.append('\n');
            }
        }
        return sb.toString();
    }

    @Override
    public boolean equals(Object o) {
        if (!(o instanceof Maze)) return false;
        Maze that = (Maze)o;
        return this.toString().equals(that.toString());
    }

    @Override
    public int hashCode() { return this.toString().hashCode(); }

    boolean isPerfect() {
        UnionFind uf = new UnionFind(height * width);
        for (int i = 0; i < height; ++i) {
            for (int j = 0; j < width-1; ++j) {
                if (grid[i][j].hasPassageTo(grid[i][j+1])) {
                    if (uf.sameClass(coordToInt(i,j), coordToInt(i,j+1))) return false;
                    uf.union(coordToInt(i,j), coordToInt(i,j+1));
                }
            }
            if (i == height-1) continue;
            for (int j = 0; j < width; ++j) {
                if (grid[i][j].hasPassageTo(grid[i+1][j])) {
                    if (uf.sameClass(coordToInt(i,j), coordToInt(i+1,j))) return false;
                    uf.union(coordToInt(i,j), coordToInt(i+1,j));
                }
            }
        }
        return uf.getSize(0) == height * width;
    }

    void clearMarks() {
        for (Cell[] row : grid)
            for (Cell c : row)
                c.setMarked(false);
    }
}

## Code generation
The code you just downloaded is organized into several classes. Here are their main functions:
- The class Maze models a maze. It contains the 2-dimensional array grid containing elements
of type Cell. The size of the array is heightxwidth. </br>
\- The method `getCell(i, j)` allows to access the box with the coordinates __( i, j)__. </br>
\- The method `getFirstCell()` returns the first box, that is, the box with coordinates
(0,0). </br>

- The class Cell models a square in a maze. Each square therefore has four possible neighboring
squares: north, east, south, and west. A wall can block the path to each of these neighboring
squares.</br>
\- The method `getNeighbors(boolean ignoreWalls)` returns a list of neighboring cells.
If the argument `ignoreWalls` is `true`, then all neighboring cells are returned. If the
argument `ignoreWalls` is `false`, then only cells to which a passage exists are returned.</br>
\- The method `breakWall(Cell c)` creates a passage from the current square to the square
c given as an argument. It therefore removes a wall.</br>
\- A mark can be placed on a square with the method `setMarked`. The existence of a mark
can be tested with the method `isMarked`.

# 1 Solution of a maze by backtracking

In this first question, we want to find a path in a given maze, going from the northwest corner to
the southeast corner, that is, from the cell with coordinates (0,0) to the exit, in our case the one
with coordinates __(height −1,width −1)__. The `isExit` method in the `Cell` class can be useful: it
returns `true` if and only if the current cell is the exit. The path must be indicated with marks on
the cells. More precisely, a call to the `isMarked` method must return `true` for all cells on the path
and `false` for all cells that are not on the path.
For example, in the 5×5 maze

![maze](5x5.png)

the single path from northwest to southeast is

![maze](5x5_1.png)

where a marked box is distinguished by a red circle.

Complete the method `searchPath()` of the class `ExtendedCell`, so that:

\(1) it returns `true` if there exists between this and the exited cell a path not passing through any
marked cell, and marks the cells of such a path;

\(2) it returns `false` otherwise, and in this case, it does not modify any mark.

To find a path from the entrance to the exit in the maze, it will be enough to call the method on
the Cell instance representing the (0,0) box. This is what is done in the tests.

You may notice that the method starts with the call `maze.slow()`. This is used to slow down the
pathfinding animation so that it is observable to the naked eye.

Test your method with Test1

# 2 Maze generation by recursive backtracking

In the rest of the assignment, we will work on maze generation . As with the mazes in Question 1,
they must be perfect, that is, they must have the property that there is a unique path between each
pair of squares. Initially, all the squares are isolated (i.e., there are walls in all four directions).

Recursive backtracking for generation has the same general structure as pathfinding. One difference
is that it is not necessary to mark any cells. Another difference is the condition for the recursive 
call: we want to continue only if the neighbor is isolated. Otherwise, it would lead to creating a
cycle and the maze would not be perfect.

To avoid always ending up in the same maze, we want to run through the directions in random
order. To do this, we can use the static method shuffle of the auxiliary class Collections which
takes a list and randomly permutes it.

So we have the following simple structure for our algorithm in the `generateRec` method of the
class `ExtendedCell`:
- For all adjacent squares in random order: if the neighbor is isolated, create a connection (by
removing the wall with `breakWall`) and make a recursive call with the neighbor.

Test your method with Test2

# 3 Maze generation by iterative backtracking

We will now translate the recursive generation algorithm into an iterative version. This means that
we exchange the call stack for a linked list that contains the coordinates of the cells we have started
processing.

An iterative approach can have several advantages: less space on the call stack and more control
over the order in which cells are visited. The call stack always chooses the most recent element.
With an explicit list, this behavior can be changed.

The class `Bag` (implemented in the `Util.java file`, which you don’t need to look at) offers several
ways to choose the next element (of type `Cell`) from a list. The four different ways available are:
`NEWEST`(we choose the newest element), `OLDEST`(for the oldest element), `MIDDLE`(the element in the
middle of the list) and `RANDOM`(a random element from the list). For each of the four options, the
method `peek()` returns the element selected according to the choice. The method `pop()` removes
this element. Both methods assume a non-empty list. To test if the list is indeed empty, we can
use the method `isEmpty()`. The method `add(Cell c)` allows adding a new element c to the end
of the list. We also specify that the `peek()` and methods `pop()` are synchronized. In particular, if
a cell is selected in the list with `peek()` and other cells are added to the list, the next call to `pop()`
removes the previously selected cell.

To implement the iterative algorithm in the method `generateIter` of the class `Maze`, we can follow
the following process. The algorithm starts with a list that contains only the element (0,0), then
enters a loop, like this:

\(1) If the list is not empty, select an item from the list using the selection method. Otherwise,
finish. </br>
\(2) For the first isolated neighboring box in random order: create a passage, add the neighbor to
the list and go to step (1).</br>
\(3) If you have exhausted all directions around the current square, remove that square from the
list and return to step (1).</br>

You may notice that a choice of `NEWEST` returns to the recursive version. A choice of `RANDOM`
produces quite interesting mazes too, which is not the case for the choices `MIDDLE` or `OLDEST`.

Test your method with the Test3

# 4 Uniform generation: Wilson’s algorithm

We have just implemented random maze generation algorithms. The shape of the mazes generated
by iterative backtracking strongly depended on the choice of the method for selecting the next cell
to be processed. In particular, the generated mazes are not uniformly distributed across the set of
all perfect mazes of the given size.

In this exercise we will therefore implement a uniform generation algorithm. The generation method
chosen is called Wilson’s algorithm . It proceeds as follows:

1. Choose a box uniformly at random and mark it. </br>
2. As long as there is an unmarked square, choose an unmarked square uniformly at random,
and </br>
    (i) generate a uniform random walk from the chosen square until a marked square is en-
    countered (ignore walls to generate the walk). Remove loops from the walk: only keep
    the last exit from each square.  
    (ii) mark all the boxes and create the connections (by deleting the walls with `breakWall`)
    along the generated step.

Here is the algorithm in mid-execution:

<div>
<img src="wilsons.png" width="500"/>
</div>

Implement Wilson’s algorithm by extending the method `generateWilson()` of the class `Maze`. To
draw random uniform boxes successively, simply put all the boxes in a linked list and shuffle it. We
can add to the class `Cell` a field `Cell` next, to keep track of the direction taken by the random
walk the last time it leaves a box.

Test your method with the Test4

### Your code here

In [4]:
class ExtendedCell extends Cell {
	
	public ExtendedCell(Maze maze) {
		super(maze);
	}

    /**
     * private boolean isMarked;
     * private boolean isExit;
     * private ArrayList<Cell> neighbors;
     * private ArrayList<Boolean> walls;
     * protected Maze maze;
     */

    /*
     *   N      2
     * W   E  3   1
     *   S      0
	*/

	// Question 1

	/**
	 * Test if there is a path from the current cell to an exit
	 * 
	 * @return true if there is a path from the current cell to an exit
	 */
	boolean searchPath() {
		maze.slow(); // slow down the search animation (to help debugging)

        this.setMarked(true); 
        
        if (this.isIsolated()) return false; // no neighbors, no path
        if (this.isExit()) return true; // found an exit
        
        for (Cell neighbor : this.getNeighbors(false)) { 
            if (neighbor.isMarked()) continue;
            if (neighbor.searchPath()){
                return true;
            }
            else{
                neighbor.setMarked(false);
            }
        }
		return false;
	}

	// Question 2

	/**
	 * generate a perfect maze using recursive backtracking
	 */
	void generateRec() {
		maze.slow();

        if (this.isExit()) return;

        var neighbors = this.getNeighbors(true);
        Collections.shuffle(neighbors);
        
        for (Cell neighbor : neighbors) { 
            if (!neighbor.isIsolated()) continue;           
            this.breakWall(neighbor);
            neighbor.generateRec();
        }
	}

}
// Redefine Maze methods
class ExtendedMaze extends Maze {

    /*
     * protected  int height, width;
     * protected  Cell[][] grid;
     * private MazeFrame frame;
     * private static final int step = 20;
     */

    ExtendedMaze(String path) throws IOException {
        super(path); // call Maze(String path)
    }
    
    // Constructor for maze25.txt, maze3.txt, etc.
    ExtendedMaze(String path, boolean window) throws IOException {
        super(path, window);  // delegate to Maze constructor
    }

    ExtendedMaze(int height, int width) {
        super(height, width);
    }

    ExtendedMaze(int height, int width, boolean window) {
        super(height, width, window);
    }

    ExtendedMaze(List<String> lines, boolean window) {
        super(lines, window);
    }
    
    @Override
    void generateIter(int selectionMethod) {
        /*
         * (1) If the list is not empty, select an item from the list using the selection method. 
         *     Otherwise,finish.
         * 
         * (2) For the first isolated neighboring box in random order: 
         *     create a passage, add the neighbor to the list and go to step (1).
         * 
         * (3) If you have exhausted all directions around the current square, 
         *     remove that square from the list and return to step (1).
         */
        Bag cells = new Bag(selectionMethod);
        cells.add(getFirstCell());
        while (!cells.isEmpty()) {
            slow();

            Cell choice = cells.peek();
            Cell choiceNeighbor = null;
            var neighbors = choice.getNeighbors(true);
            Collections.shuffle(neighbors);

            for(var neighbor : neighbors){
                if(!neighbor.isIsolated()) continue;
                choiceNeighbor = neighbor;
                break;
            }

            if (choiceNeighbor == null) {
                cells.pop();
                continue;
            }

            choice.breakWall(choiceNeighbor);
            cells.add(choiceNeighbor);
        }
    }

    @Override
    void generateWilson() {
        /*
            1. Choose a box uniformly at random and mark it.

            2. As long as there is an unmarked square, choose an unmarked square uniformly at random, and 
            (i) generate a uniform random walk from the chosen square until a marked square is encountered
            (ignore walls to generate the walk). 
            Remove loops from the walk: only keep the last exit from each square.  
            
            (ii) mark all the boxes and create the connections (by deleting the walls with `breakWall`)
            along the generated step.
        
            Implement Wilson’s algorithm by extending the method `generateWilson()` of the class `Maze`.
            To draw random uniform boxes successively, simply put all the boxes in a linked list and shuffle it.
            We can add to the class `Cell` a field `Cell` next, to keep track of the direction taken by the random
            walk the last time it leaves a box.
         */

        ArrayList<Cell> unmarkeds = new ArrayList<>();
        ArrayList<Cell> cells = new ArrayList<>();
        for (int i = 0; i < height; ++i) {
            for (int j = 0; j < width; ++j) {
                cells.add(grid[i][j]);
                unmarkeds.add(grid[i][j]);
            }
        }

        Cell firstMark = unmarkeds.remove((int)(Math.random() * unmarkeds.size()));
        firstMark.setMarked(true);
        
        while(!unmarkeds.isEmpty()){
            slow();

            Cell choice = unmarkeds.remove((int)(Math.random() * unmarkeds.size()));
            Cell current = choice;
            int path[] = new int[height * width];

            while(!current.isMarked()){
                var neighbors = current.getNeighbors(true);
                Collections.shuffle(neighbors);
                Cell next = neighbors.get(0);

                path[cells.indexOf(current)] = cells.indexOf(next);
                current = next;
            }

            current = choice;
            current.setMarked(true);

            while(true){
                Cell next = cells.get(path[cells.indexOf(current)]);
                current.breakWall(next);
                if (next.isMarked()) break;
                next.setMarked(true);
                unmarkeds.remove(next);
                current = next;
            }          
        }
    }
}

### Test 1

In [5]:
System.out.print("testing searchPath() with 25x25 maze... ");

Maze m25 = new ExtendedMaze("maze25.txt");
m25.getFirstCell().searchPath();

Maze m25sol = new ExtendedMaze("maze25sol.txt", false);
assert(m25.equals(m25sol)) : "not the correct solution";

System.out.println("\t\t[OK]");

testing searchPath() with 25x25 maze... 		[OK]


### Test 2

In [6]:
private static void assertPerfect(Maze m) {
	assert(m.isPerfect()) : "maze is not perfect";
}

System.out.print("testing generateRec() for a 5x5 maze... ");
Maze m = new ExtendedMaze(5, 5);
m.getFirstCell().generateRec();
assertPerfect(m);
System.out.println("\t\t[OK]");

System.out.print("testing generateRec() with more mazes of size 5x5... ");
for(int k = 0; k < 10; ++k) {
    m = new ExtendedMaze(5, 5, false);
    m.getFirstCell().generateRec();
    assertPerfect(m);
}
System.out.println("\t[OK]");

System.out.print("testing generateRec() for a 25x25 maze... ");
m = new ExtendedMaze(25, 25);
m.getFirstCell().generateRec();
assertPerfect(m);
Maze m1 = new ExtendedMaze(25, 25, false);
m1.getFirstCell().generateRec();
Maze m2 = new ExtendedMaze(25, 25, false);
m2.getFirstCell().generateRec();
assert(!m1.equals(m2)) : "Maze generation is not random.";
System.out.println("\t\t[OK]");

testing generateRec() for a 5x5 maze... 		[OK]
testing generateRec() with more mazes of size 5x5... 	[OK]
testing generateRec() for a 25x25 maze... 		[OK]


### Test 3

In [7]:
import java.util.LinkedList;

private static void assertPerfect(Maze m) {
    assert(m.isPerfect()) : "maze is not perfect";
}

System.out.print("testing generateIter() with method Newest... ");
Maze m = new ExtendedMaze(25, 25);
m.generateIter(Bag.NEWEST);
assertPerfect(m);
for(int k = 0; k < 10; ++k) {
    m = new ExtendedMaze(5, 5, false);
    m.generateIter(Bag.NEWEST);
    assertPerfect(m);
}
Maze m1 = new ExtendedMaze(25, 25, false);
m1.generateIter(Bag.NEWEST);
Maze m2 = new ExtendedMaze(25, 25, false);
m2.generateIter(Bag.NEWEST);
assert(!m1.equals(m2)) : "Maze generation is not random.";
System.out.println("\t\t[OK]");


System.out.print("testing generateIter() with method Random... ");
m = new ExtendedMaze(25, 25);
m.generateIter(Bag.RANDOM);
assertPerfect(m);
for(int k = 0; k < 10; ++k) {
    m = new ExtendedMaze(5, 5, false);
    m.generateIter(Bag.RANDOM);
    assertPerfect(m);
}
m1 = new ExtendedMaze(25, 25, false);
m1.generateIter(Bag.RANDOM);
m2 = new ExtendedMaze(25, 25, false);
m2.generateIter(Bag.RANDOM);
assert(!m1.equals(m2)) : "Maze generation is not random.";
System.out.println("\t\t[OK]");

System.out.print("testing generateIter() with method Middle... ");
m = new ExtendedMaze(25, 25);
m.generateIter(Bag.MIDDLE);
assertPerfect(m);
for(int k = 0; k < 10; ++k) {
    m = new ExtendedMaze(5, 5, false);
    m.generateIter(Bag.MIDDLE);
    assertPerfect(m);
}
System.out.println("\t\t[OK]");

System.out.print("testing generateIter() with method Oldest... ");
m = new ExtendedMaze(25, 25);
m.generateIter(Bag.OLDEST);
assertPerfect(m);
for(int k = 0; k < 10; ++k) {
    m = new ExtendedMaze(5, 5, false);
    m.generateIter(Bag.OLDEST);
    assertPerfect(m);
}
System.out.println("\t\t[OK]");


LinkedList<String> l = new LinkedList<String>();
LinkedList<String> l2 = new LinkedList<String>();
l.add("2");
l.add("2");
l.add("E");
l.add("WS");
l.add("E");
l.add("WN");
l2.add("2");
l2.add("2");
l2.add("S");
l2.add("S");
l2.add("NE");
l2.add("NW");
Maze mTest = new ExtendedMaze(l,false);
Maze mTest2 = new ExtendedMaze(l2, false);	

m = new ExtendedMaze(2,2, false);
m.generateIter(Bag.NEWEST);

System.out.print("Additional test... ");
assert(m.equals(mTest) || m.equals(mTest2)) : "Error: when a connection is created, exit the loop and return to step 1.";		
System.out.println("\t\t\t\t\t[OK]");


testing generateIter() with method Newest... 		[OK]
testing generateIter() with method Random... 		[OK]
testing generateIter() with method Middle... 		[OK]
testing generateIter() with method Oldest... 		[OK]
Additional test... 					[OK]


### Test 4

In [8]:
private static void assertPerfect(Maze m) {
    assert(m.isPerfect()) : "maze is not perfect";
}

System.out.print("testing whether generateWilson() generates perfect mazes... ");
Maze m = new ExtendedMaze(25, 25);
m.generateWilson();
assertPerfect(m);
for(int k = 0; k < 10; ++k) {
    m = new ExtendedMaze(5, 5, false);
    m.generateWilson();
    assertPerfect(m);
}
System.out.println("\t[OK]");

System.out.print("testing uniformity of generateWilson()... ");
Maze m3 = new ExtendedMaze("maze3.txt", false);
int cnt = 0;

// Chernoff bound parameters
final int nbMazes = 192;
final int K = 1000000;
final double mu = K / (double) nbMazes;
final double eps = 0.01;
final double delta = Math.sqrt( 3*Math.log(2/eps)/mu );

for(int k = 0; k < K; ++k) {
    m = new ExtendedMaze(3, 3, false);
    m.generateWilson();
    m.clearMarks();
    if(m.equals(m3))
        ++cnt;
}

assert( cnt > (1-delta)*mu && cnt < (1+delta)*mu ) : "not uniform";
System.out.println("\t\t\t[OK]");

testing whether generateWilson() generates perfect mazes... 	[OK]
testing uniformity of generateWilson()... 			[OK]


## note

- The application should launch in a GUI form format, if it didn't launch, contact the TA.
- The application can be closed using **x** on the top right
- If you see `Kernel Restarting` popout box, no need to worry, just close it as normal, restart kernel and run everything again.
 
**Remember:** If you changed the code, you need to restart the kernel and rerun everything again or else you might experience unexpected bugs.